In [1]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=3

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=3


In [2]:
import pandas as pd
import numpy as np
import torch
import optuna

from functools import partial

from sklearn.model_selection import StratifiedGroupKFold
from sklearn.base import BaseEstimator, clone, ClassifierMixin
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from catboost import CatBoostClassifier, Pool
from transformers import AutoTokenizer, AutoModel

from ax import optimize


import wandb

## Загрузка данных

In [3]:
np.random.seed(59)
X_train = pd.read_csv("data/my/before_feature_engineering/train.csv", sep=",", header=0).drop(columns="Unnamed: 0").iloc[:-300]
y_train = pd.read_csv("data/my/before_feature_engineering/train_labels.csv", sep=",", header=0).drop(columns="Unnamed: 0").iloc[:-300]
X_train_tr = pd.read_csv("data/my/after_feature_engineering/train.csv", sep=",", header=0).drop(columns="Unnamed: 0").iloc[:-300]
groups_train = pd.read_csv("data/my/before_feature_engineering/train_groups.csv", sep=",", header=0).drop(columns="Unnamed: 0")

permutation = np.random.permutation(X_train.shape[0])
X_train = X_train.iloc[permutation].reset_index(drop=True)
X_train_tr = X_train_tr.iloc[permutation].reset_index(drop=True)
y_train = y_train.iloc[permutation].reset_index(drop=True)
groups_train = groups_train.iloc[permutation].reset_index(drop=True)["0"]

X_test = pd.read_csv("data/my/before_feature_engineering/test.csv", sep=",", header=0).drop(columns="Unnamed: 0")
X_test_tr = pd.read_csv("data/my/after_feature_engineering/test.csv", sep=",", header=0).drop(columns="Unnamed: 0")
y_test = pd.read_csv("data/my/before_feature_engineering/test_labels.csv", sep=",", header=0).drop(columns="Unnamed: 0")

y_train = y_train.fraudulent
y_test = y_test.fraudulent

## Эксперименты с моделями без тюнинга гиперпараметров

### Вспомогательные функции

In [4]:

class MajorityVoteClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, estimators):
        self.estimators = estimators

    def predict(self, X):
        predictions = np.array([estimator.predict(X) for estimator in self.estimators])
        sum_predictions = np.sum(predictions.T, axis=1)
        majority = (sum_predictions > (len(self.estimators) / 2)).astype(int)
        return majority

def cross_validate_and_log_metrics(model, X, y, groups, cv_class=StratifiedGroupKFold, n_splits=5, test_share_range=(0.1, 0.3)):
    cv = cv_class(n_splits=n_splits, shuffle=True, random_state=42)
    fold_metrics = {'f1': [], 'precision': [], 'recall': []}
    models = []
    for fold, (train_idx, val_idx) in enumerate(cv.split(X, y, groups=groups)):
        if isinstance(X, pd.DataFrame):
            X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        elif isinstance(X, np.ndarray):
            X_train, X_val = X[train_idx], X[val_idx]
        else:
            raise TypeError('Invalid type for X argument')
        if isinstance(y, pd.DataFrame) or isinstance(y, pd.Series):
            y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
        elif isinstance(y, np.ndarray):
            y_train, y_val = y[train_idx], y[val_idx]
        else:
            raise TypeError('Invalid type for y argument')
        test_share = X_val.shape[0] / (X_val.shape[0] + X_train.shape[0])
        if test_share < test_share_range[0] or test_share > test_share_range[1]:
            continue
    
        model_new = clone(model)
        model_new.fit(X_train, y_train)
        models.append(model_new)
        y_pred = model_new.predict(X_val)
        
        fold_metrics['f1'].append(f1_score(y_val, y_pred, zero_division=0))
        fold_metrics['precision'].append(precision_score(y_val, y_pred, zero_division=0))
        fold_metrics['recall'].append(recall_score(y_val, y_pred, zero_division=0))
        
        wandb.log({
            'fold': fold + 1,
            'fold_f1': fold_metrics['f1'][-1],
            'fold_precision': fold_metrics['precision'][-1],
            'fold_recall': fold_metrics['recall'][-1]
        })
    
    mean_metrics = {
        'mean_f1': np.mean(fold_metrics['f1']),
        'mean_precision': np.mean(fold_metrics['precision']),
        'mean_recall': np.mean(fold_metrics['recall'])
    }
    
    wandb.log({
        **mean_metrics,
        'status': 'completed'
    })
    
    return mean_metrics, MajorityVoteClassifier(models)

def calc_and_print_metrics(model, X, y, is_test=True):
    y_pred = model.predict(X)
    calc_set_string = "Test" if is_test else "Train"
    print(f"{calc_set_string} F1-Score: {f1_score(y, y_pred):.3f}")
    print(f"{calc_set_string} Precision: {precision_score(y, y_pred):.3f}")
    print(f"{calc_set_string} Recall: {recall_score(y, y_pred):.3f}")
            

### Эксперименты

#### SVM

В качестве бейзлайна, мы брали логистическую регрессию, поэтому в качестве первой модели сейчас можно попробовать ядровой SVM, так как он позволит уловить более сложные закономерности в данных за счет нелинейности в ядре.

In [48]:
model_svm = SVC(
    class_weight='balanced',
    random_state=42,
    max_iter=5000,
    kernel="rbf"
)

n_folds = 5

with wandb.init(project="job-fake-prediction", 
          config={
              "model_type": "svm",
              "validation": "stratified_group_kfold",
              "k_folds": n_folds
          }):
    metrics, averaged_model_svm = cross_validate_and_log_metrics(model_svm, X_train_tr, y_train, groups_train, n_splits=n_folds)
    calc_and_print_metrics(averaged_model_svm, X_train_tr, y_train, is_test=False)
    calc_and_print_metrics(averaged_model_svm, X_test_tr, y_test)

Train F1-Score: 0.630
Train Precision: 0.460
Train Recall: 1.000
Test F1-Score: 0.277
Test Precision: 0.528
Test Recall: 0.188


fold,▁▅█
fold_f1,█▁▆
fold_precision,▄▁█
fold_recall,█▂▁
mean_f1,▁
mean_precision,▁
mean_recall,▁
fold,5
fold_f1,0.34286
fold_precision,0.75
fold_recall,0.22222


Скрины метрик (step здесь имеет смысл очередного сплита кросс-валидации):
![](images/svm/f1.png)
![](images/svm/precision.png)
![](images/svm/recall.png)

Видно, что модель перформит лучше бейзлайна (f1_score 0.27 против 0.23) -- нелинейности помогли

#### Случайаный лес

Теперь попробуем ещё усложнить модель, взяв случайный лес -- может быть, качество еще вырастет.

In [49]:
model_rf = RandomForestClassifier(
    class_weight='balanced',
    random_state=42,
)

with wandb.init(project="job-fake-prediction", 
          config={
              "model_type": "random_forest",
              "validation": "stratified_group_kfold",
              "k_folds": n_folds
          }):
    metrics, averaged_model_rf = cross_validate_and_log_metrics(model_rf, X_train_tr, y_train, groups_train, n_splits=n_folds)
    calc_and_print_metrics(averaged_model_rf, X_train_tr, y_train, is_test=False)
    calc_and_print_metrics(averaged_model_rf, X_test_tr, y_test)

Train F1-Score: 1.000
Train Precision: 1.000
Train Recall: 1.000
Test F1-Score: 0.112
Test Precision: 1.000
Test Recall: 0.059


fold,▁▅█
fold_f1,█▃▁
fold_precision,▁▁▁
fold_recall,█▂▁
mean_f1,▁
mean_precision,▁
mean_recall,▁
fold,5
fold_f1,0.10526
fold_precision,1
fold_recall,0.05556


Скрины метрик (step здесь имеет смысл очередного сплита кросс-валидации):
![](images/random_forest/f1.png)
![](images/random_forest/precision.png)
![](images/random_forest/recall.png)

Видно, что модель сильно переобучилась и показывает себя хуже себя, чем логистическая регрессия (бейзлайн). Может быть, тюнинг гиперпараметров поможет её регулязировать.

#### Градиентный бустинг

Известно, что catboost хорошо работает с текстовыми признаками, поэтому обучим его на исходных текстовых признаках после их предобработки. За счет нативной работы с текстовыми признаками, мы ожидаем, что результат может потенциально превзойти предыдущие эксперименты по качеству.

In [22]:
raw_text_features = ["title_processed", "description_processed", "company_profile_processed"]
X_train_cb = X_train[raw_text_features].fillna('')
X_test_cb = X_test[raw_text_features].fillna('')

In [112]:
class MetricsLoggerCallback:
    def after_iteration(self, info):
        test_metrics = info.metrics["validation"]
        wandb.log({
            'f1': test_metrics['F1'][-1],
            'precision': test_metrics['Precision:use_weights=false'][-1],
            'recall': test_metrics['Recall:use_weights=false'][-1],
        })
        return True

In [113]:
model_cb = CatBoostClassifier(
        iterations=100,  
        loss_function='Logloss',
        eval_metric='F1',
        custom_metric=['Precision', 'Recall'],
        class_weights=[1, 5],
        text_features=raw_text_features,
        verbose=False,
        use_best_model=False,
        early_stopping_rounds=None,
        
    )


with wandb.init(project="job-fake-prediction", 
          config={
              "model_type": "catboost",
          }):
    model_cb.fit(
        X_train_cb, y_train, 
        callbacks=[MetricsLoggerCallback()],
        eval_set=[Pool(X_test_cb, y_test, text_features=raw_text_features)]
    )
    calc_and_print_metrics(model_cb, X_train_cb, y_train, is_test=False)
    calc_and_print_metrics(model_cb, X_test_cb, y_test)


Train F1-Score: 0.801
Train Precision: 0.705
Train Recall: 0.928
Test F1-Score: 0.446
Test Precision: 1.000
Test Recall: 0.287


f1,▁▁██▇▇▇▇▆▁▃▃▃▃▃▅▅▆▅▅▅▅▅▅▅▅▅▄▄▄▄▄▄▃▃▄▅▄▅▄
precision,▁▄██████████████████████████████████████
recall,▁█▅▇▇▂▂▅▃▃▁▅▅▅▅▆▆▅▅▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▄▅▄▅▄▄
f1,0.44615
precision,1
recall,0.28713


Скрины метрик на тесте:

![](images/catboost/f1.png)
![](images/catboost/precision.png)
![](images/catboost/recall.png)

Видно, что модель переобучилась, но при этом качество все равно существенно улучшилось по сравнению со всеми предыдущими экспериментами.

#### BERT + логистическая регрессия

Ранее мы обрабатывали текст "классическими" методами (tf-idf, эвристики катбуста). Кажется, что контекстуальный эмбеддинг BERTа будет менее зашумлен и нести больше информации о тексте, что может нам помочь улучшить качество детекции фрода.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name).to(device)

def get_single_embedding(text):
    inputs = tokenizer(
        text,
        padding=True,
        truncation=True,
        max_length=512,
        return_tensors="pt"
    ).to(device)
    
    with torch.no_grad():
        outputs = model(**inputs)
    
    return outputs.last_hidden_state[:, 0].cpu().numpy()

def get_all_embeddings(df):
    title_embs = []
    desc_embs = []
    profile_embs = []
    
    for _, row in df.iterrows():
        title_embs.append(get_single_embedding(str(row['title'])))
        desc_embs.append(get_single_embedding(str(row['description'])))
        profile_embs.append(get_single_embedding(str(row['company_profile'])))
    
    return np.hstack([
        np.vstack(title_embs),
        np.vstack(desc_embs),
        np.vstack(profile_embs)
    ])

preprocessor = StandardScaler()
X_train_embeds = pd.DataFrame(preprocessor.fit_transform(get_all_embeddings(X_train)))
X_test_embeds = pd.DataFrame(preprocessor.transform(get_all_embeddings(X_test)))

/home/aksenovan/my_env/lib/python3.10/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [73]:
model_bert_logreg = LogisticRegression(
    class_weight='balanced',
    random_state=42,
    max_iter=1000
)

with wandb.init(project="job-fake-prediction", 
          config={
              "model_type": "bert+logreg",
              "validation": "stratified_group_kfold",
              "k_folds": n_folds
          }):
    metrics, averaged_model_bert_logreg = cross_validate_and_log_metrics(model_bert_logreg, X_train_embeds, y_train, groups_train, n_splits=n_folds)
    calc_and_print_metrics(averaged_model_bert_logreg, X_train_embeds, y_train, is_test=False)
    calc_and_print_metrics(averaged_model_bert_logreg, X_test_embeds, y_test)

Train F1-Score: 1.000
Train Precision: 1.000
Train Recall: 1.000
Test F1-Score: 0.512
Test Precision: 0.620
Test Recall: 0.436


fold,▁▅█
fold_f1,▁█▆
fold_precision,▁▇█
fold_recall,▁█▃
mean_f1,▁
mean_precision,▁
mean_recall,▁
fold,5
fold_f1,0.12281
fold_precision,0.11667
fold_recall,0.12963


Скрины метрик на валидации (step здесь имеет смысл очередного сплита кросс-валидации):
![](images/bert/f1.png)
![](images/bert/precision.png)
![](images/bert/recall.png)

Видно, что качество на тесте существенно увеличилось. Однако отметим, что качество на валидации очень низкое -- думаю, это связано с утечкой в данных. Скорее всего фродеры составляют объявления по некоторму шаблону, поэтому объявления одного фродера очень близки в пространстве эмбеддингов берта. В результате в тесте модель видит похожий эмбеддинг и выдает ответ, который она видела в трейне. Чтобы это поправить, я пробовал регуляризовать модель, уменьшая размерность с помощью PCA, но это, к сожалению, не помогло.

## Эксперименты с моделями с тюнингом гиперпараметров

Случайный лес показал слишком низкое качество, а линейная регрессия с бертом чересчур хорошее (из-за утечки, вероятно), поэтому здесь подберем гиперпараметры для Catboost и SVM. 

Для каждого из них воспользуемся тремя методами/фрейморками:
- gridsearch
- optuna
- ax

Напишем helper-функции и потюним гиперпараметры

In [16]:
from sklearn.model_selection import ParameterGrid

def grid_search(
    X, y, groups, model_class, param_grid,
    cv_class=StratifiedGroupKFold, n_splits=5,
    test_share_range=(0.1, 0.3),
):
    all_results = []
    best_score = -1
    best_params = None

    for params in ParameterGrid(param_grid):
        cleaned_params = params.copy()

        model = model_class(**cleaned_params)
        
        mean_metrics, _ = cross_validate_and_log_metrics(
            model, X, y, groups,
            cv_class=cv_class,
            n_splits=n_splits,
            test_share_range=test_share_range
        )

        current_score = mean_metrics["mean_f1"]
        all_results.append({
            "params": cleaned_params,
            "metrics": mean_metrics
        })

        if current_score > best_score:
            best_score = current_score
            best_params = cleaned_params

    return best_params, all_results



def tune_hyperparameters_optuna(
    X, y, groups, model_class, param_sampler,
    cv_class=StratifiedGroupKFold, n_splits=5, 
    test_share_range=(0.1, 0.3), n_trials=100,
    direction="maximize"
):
    
    def objective(trial):
        params = param_sampler(trial)
        
        model = model_class(**params)
        
        mean_metrics, _ = cross_validate_and_log_metrics(
            model, X, y, groups,
            cv_class=cv_class,
            n_splits=n_splits,
            test_share_range=test_share_range
        )
        
        return mean_metrics["mean_f1"]

    study = optuna.create_study(direction=direction)
    study.optimize(objective, n_trials=n_trials)

    return study.best_params, study

def tune_hyperparameters_ax(
    X, y, groups, model_class, param_config,
    cv_class=StratifiedGroupKFold, n_splits=5,
    test_share_range=(0.1, 0.3), n_trials=100,
    direction="maximize"
):  
    def evaluation_function(params):
        cleaned_params = {
            k: (v if v != "None" else None) 
            for k, v in params.items()
        }
        
        
        model = model_class(**cleaned_params)
        
        mean_metrics, _ = cross_validate_and_log_metrics(
            model, X, y, groups,
            cv_class=cv_class,
            n_splits=n_splits,
            test_share_range=test_share_range
        )
        
        
        return {"mean_f1": (mean_metrics["mean_f1"], 0.0)}

    best_params, best_values, experiment, _ = optimize(
        parameters=param_config,
        evaluation_function=evaluation_function,
        objective_name="mean_f1",
        minimize=direction != "maximize",
        total_trials=n_trials,
    )

    return best_params, experiment

#### SVM

##### Gridsearch

In [13]:
svm_cls = partial(
    SVC, 
    random_state=42,
    max_iter=5000,
    class_weight="balanced"
)

In [14]:
param_grid = {
    "C": [0.1, 1, 10],
    "kernel": ["linear", "rbf", "poly", "sigmoid"],
}

with wandb.init(project="job-fake-prediction", 
          config={
              "model_type": "svm_gridsearch",
              "validation": "stratified_group_kfold",
              "k_folds": n_folds
          }):
    best_params, all_results = grid_search(
        X_train_tr, y_train, groups_train,
        model_class=svm_cls,
        param_grid=param_grid,
    )

svm_model_gs = svm_cls(**best_params).fit(X_train_tr, y_train)
calc_and_print_metrics(svm_model_gs, X_train_tr, y_train, is_test=False)
calc_and_print_metrics(svm_model_gs, X_test_tr, y_test)

/home/aksenovan/my_env/lib/python3.10/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/home/aksenovan/my_env/lib/python3.10/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/home/aksenovan/my_env/lib/python3.10/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/home/aksenovan/my_env/lib/python3.10/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/home/aksenovan/my_env/lib/python3.10/site-packages/sklearn/svm/_base.py:297: Convergenc

fold,▁▅█▁▅█▁▅█▁▅█▁▅█▁▅█▁▅█▁▅█▁▅█▁▅█▁▅█▁▅█
fold_f1,▃▁▅▅▂▆▁▁▂▂▂▄▂▁▁▇▃▆█▄▄▂▂▃▁▁▂█▄▆█▃▄▂▁▃
fold_precision,▂▁▂▂▁▆▁▁▁▁▁▂▁▁▁▄▂▇▅▂▂▁▁▂▁▁▂▅▃█▅▂▃▁▁▂
fold_recall,█▂▅▄▄▂███▄▄▃▅▂▁▄▃▂▄▄▂▄▆▃▁▂▂▄▃▂▄▃▂▄▅▃
mean_f1,▃▅▁▃▁▇▆▂▁█▆▂
mean_precision,▂▅▁▂▁▆▄▂▁█▄▁
mean_recall,▄▃█▄▂▃▃▄▁▃▂▄
fold,5
fold_f1,0.15702
fold_precision,0.10106
fold_recall,0.35185


/home/aksenovan/my_env/lib/python3.10/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Train F1-Score: 0.955
Train Precision: 0.914
Train Recall: 1.000
Test F1-Score: 0.295
Test Precision: 0.679
Test Recall: 0.188


Скрины метрик на валидации (step здесь имеет смысл очередного набора параметров):
![](images/svm_tuning/gridsearch/f1.png)
![](images/svm_tuning/gridsearch/precision.png)
![](images/svm_tuning/gridsearch/recall.png)

Видно, что нам удалось улучшить f1 в сравнении с исходным запуском (0.29 против 0.27).

##### Optuna

In [17]:
def svm_param_sampler(trial):
    return {
        "C": trial.suggest_float("C", 1e-3, 1e3, log=True),
        "kernel": trial.suggest_categorical("kernel", ["linear", "rbf", "sigmoid", "poly"]),
    }

with wandb.init(project="job-fake-prediction", 
          config={
              "model_type": "svm_optuna_tunning",
              "validation": "stratified_group_kfold",
              "k_folds": n_folds
          }):
    best_params, study = tune_hyperparameters_optuna(
        X_train_tr, y_train, groups_train,
        model_class=svm_cls,
        param_sampler=svm_param_sampler,
        n_trials=50
    )

optuna_model_svm = svm_cls(**best_params).fit(X_train_tr, y_train)
calc_and_print_metrics(optuna_model_svm, X_train_tr, y_train, is_test=False)
calc_and_print_metrics(optuna_model_svm, X_test_tr, y_test)

[I 2025-03-24 20:09:48,133] A new study created in memory with name: no-name-9c65f319-a5c6-4028-9520-f09941cd2325
/home/aksenovan/my_env/lib/python3.10/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/home/aksenovan/my_env/lib/python3.10/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
/home/aksenovan/my_env/lib/python3.10/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
[I 2025-03-24 20:09:58,402] Trial 0 finished with value: 0.08681554008167426 and parameters: {'C': 593.2860624025528, 'kernel': 'sigmoid'}. Best is trial 0 with value: 0.08681554008167426.
/home/aksenov

fold,▁▅█▁▁█▁▁▅█▁▅█▅██▁█▅▁▁▅██▁█▅▅▅█▅▅██▅█▁▅▁▅
fold_f1,▂▁▃▄▁▃▁▂▃▁▂▄▂▆▆█▆▇█▄▇▄▇▆▁▄▇▇▂▃▂█▄█▆█▆█▄▄
fold_precision,▂▁▁▁▁▁▁▆▂▆▂▁▁▅▅▅█▄▅▂▂▃▁▂█▄█▁▁▇▆▁▄▂▃██▁▅▄
fold_recall,▄▁▃▂▂▁▁▂▂█▂▄▂▄▂▂▄▄▂▄▄▂▂▂▂▃▆▂▄▂▆▂▁▂▃▂▄▂▄▃
mean_f1,▂▄▂▂▂▁▁▂▅▅▁▆▇███▆███▆█▆▂███▂▇▂▂▆▂███▂██▇
mean_precision,▁▂▁▁▁▁▁▁▁▄▂▁▅▆███▄██▇█▅▇▅▇██▇▂▁▆▁▄▁████▄
mean_recall,▄▄▄▂▂▄▁█▂▂█▃▃▂▂▂▂▂▂▂▂▃▂▂▄▂▂▂▂▂▃▂▂▂▂▂▃▂▂▃
fold,5
fold_f1,0.22642
fold_precision,0.23077
fold_recall,0.22222


/home/aksenovan/my_env/lib/python3.10/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Train F1-Score: 0.886
Train Precision: 0.795
Train Recall: 1.000
Test F1-Score: 0.305
Test Precision: 0.667
Test Recall: 0.198


Скрины метрик на валидации (step здесь имеет смысл очередного набора параметров):
![](images/svm_tuning/optuna/f1.png)
![](images/svm_tuning/optuna/precision.png)
![](images/svm_tuning/optuna/recall.png)

Отметим, что мы ещё немного улучшили f1 на тесте в сравнении с gridsearch-подбором.

##### Ax

In [20]:
param_config = [
    {
        "name": "C",
        "type": "range",
        "bounds": [1e-3, 1e3],
        "log_scale": True
    },
    {
        "name": "kernel",
        "type": "choice",
        "values": ["linear", "rbf", "sigmoid", "poly"]
    },
    
]

with wandb.init(project="job-fake-prediction", 
          config={
              "model_type": "svm_ax_tunning",
              "validation": "stratified_group_kfold",
              "k_folds": n_folds
          }):
    best_params, experiment = tune_hyperparameters_ax(
        X_train_tr, y_train, groups_train,
        model_class=svm_cls,
        param_config=param_config,
        n_trials=30
    )



ax_model_svm = svm_cls(**best_params).fit(X_train_tr, y_train)
calc_and_print_metrics(ax_model_svm, X_train_tr, y_train, is_test=False)
calc_and_print_metrics(ax_model_svm, X_test_tr, y_test)

[INFO 03-24 20:22:26] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter C. If that is not the expected value type, you can explicitly specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 03-24 20:22:26] ax.service.utils.instantiation: Inferred value type of ParameterType.STRING for parameter kernel. If that is not the expected value type, you can explicitly specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
/home/aksenovan/my_env/lib/python3.10/site-packages/ax/service/utils/instantiation.py:258: AxParameterWarning: `is_ordered` is not specified for `ChoiceParameter` "kernel". Defaulting to `False`  since the parameter is a string with more than 2 choices.. To override this behavior (or avoid this warning), specify `is_ordered` during `ChoiceParameter` construction. Note that choice parameters with exactly 2 choices are always considered ordered and that the user-supplied `is_ordered` has no effect

fold,▅█▅█▁▅█▁█▁█▁█▁▅▅▁▅▁▅▅█▁▅█▁▅▅█▁▁█▁▅▁█▁▅▁▁
fold_f1,▁▂▃▆▂▅▂▃▆▂▄▄▆▇▄▄▃▂▄▄▄▁█▄▆▄▇▆█▄▆▆▆█▄▆▆▆▄▆
fold_precision,▁▁▁▂▅▃▃▂▂▂▂▃█▄▂▂▁▁▂▄█▄▂▄█▅▂█▂▅█▂██▄▄▄▃▂█
fold_recall,█▁▃▁▁▃▄▂▁▃▁▆▂▄▁▁▁▁█▃▁▃▁▃▁▁▃▁▃▃▁▃▁▁▃▁▃▃▁▁
mean_f1,▁▂▇▃▆▆▇▃█▇▇▂▄██▇▁██████▇▇███▇█
mean_precision,▁▁▇▂▄▄▆▂█▇▇▁▃█▇▇▁█▇██▇█▇▇█▇▇▆█
mean_recall,█▁▁▃▂▁▁▁▁▁▁▄▂▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▂▁
fold,5
fold_f1,0.33333
fold_precision,0.91667
fold_recall,0.2037


/home/aksenovan/my_env/lib/python3.10/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Train F1-Score: 0.997
Train Precision: 0.994
Train Recall: 1.000
Test F1-Score: 0.328
Test Precision: 0.667
Test Recall: 0.218


Скрины метрик на валидации (step здесь имеет смысл очередного набора параметров):
![](images/svm_tuning/ax/f1.png)
![](images/svm_tuning/ax/precision.png)
![](images/svm_tuning/ax/recall.png)

Видно, что мы получили еще более высокое качество на тесте, чем у optuna.

#### Catboost

##### Gridsearch

In [36]:
cb_cls = partial(CatBoostClassifier,
    loss_function='Logloss',
    text_features=raw_text_features,
    verbose=False,
)

In [37]:
catboost_grid_config = {
    "iterations": [100],
    "depth": [4, 8],
    "l2_leaf_reg": [1, 5],
    "auto_class_weights": ["Balanced", "SqrtBalanced"]
}


with wandb.init(project="job-fake-prediction", 
          config={
              "model_type": "catboost_gridsearch",
              "validation": "stratified_group_kfold",
              "k_folds": n_folds
          }):
    best_params, all_results = grid_search(
        X_train_cb, y_train, groups_train,
        model_class=cb_cls,
        param_grid=catboost_grid_config,
    )

cb_model_gs = cb_cls(**best_params).fit(X_train_cb, y_train)
calc_and_print_metrics(cb_model_gs, X_train_cb, y_train, is_test=False)
calc_and_print_metrics(cb_model_gs, X_test_cb, y_test)

fold,▁▅█▁▅█▁▅█▁▅█▁▅█▁▅█▁▅█▁▅█
fold_f1,▆▆▂▆▇▂▇▇▂▇▇▂█▇▁█▄▁█▇▂█▇▂
fold_precision,▃▁▅▃▃▃▄▃▃▄▃▃████████████
fold_recall,▇█▂▇█▂▇█▂▇█▂▇▆▁▇▃▁▇▆▁▇▆▁
mean_f1,▃▆▆▆▇▁██
mean_precision,▁▁▂▂████
mean_recall,████▄▁▄▄
fold,5
fold_f1,0.28571
fold_precision,1
fold_recall,0.16667


Train F1-Score: 0.732
Train Precision: 0.637
Train Recall: 0.861
Test F1-Score: 0.397
Test Precision: 1.000
Test Recall: 0.248


Скрины метрик на валидации (step здесь имеет смысл очередного набора параметров):
![](images/catboost_tuning/gridsearch/f1.png)
![](images/catboost_tuning/gridsearch/precision.png)
![](images/catboost_tuning/gridsearch/recall.png)

Видно, что подбором гиперпараметров мы только ухудшили f1-меру на тесте. Думаю, это связано с тем, что мы "переобучились" под валидацию.

##### Optuna

In [31]:
def catboost_param_sampler(trial):
    return {
        "iterations": trial.suggest_int("iterations", 50, 250),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2, log=True),
        "depth": trial.suggest_int("depth", 4, 10),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1e-2, 10, log=True),
        "auto_class_weights": trial.suggest_categorical("auto_class_weights", 
                                                 ["Balanced", "SqrtBalanced"])
    }

with wandb.init(project="job-fake-prediction", 
          config={
              "model_type": "catboost_optuna",
              "validation": "stratified_group_kfold",
              "k_folds": n_folds
          }):
    best_params, study = tune_hyperparameters_optuna(
        X_train_cb, y_train, groups_train,
        model_class=cb_cls,
        param_sampler=catboost_param_sampler,
        n_trials=15
    )

cb_model_optuna = cb_cls(**best_params).fit(X_train_cb, y_train)
calc_and_print_metrics(cb_model_optuna, X_train_cb, y_train, is_test=False)
calc_and_print_metrics(cb_model_optuna, X_test_cb, y_test)

[I 2025-03-24 20:40:31,040] A new study created in memory with name: no-name-7cebd49f-1a57-4d96-902f-ccc63b653087
[I 2025-03-24 20:41:27,572] Trial 0 finished with value: 0.41130565938750085 and parameters: {'iterations': 131, 'learning_rate': 0.09485036555910548, 'depth': 7, 'l2_leaf_reg': 0.060936787353252315, 'auto_class_weights': 'Balanced'}. Best is trial 0 with value: 0.41130565938750085.
[I 2025-03-24 20:45:26,901] Trial 1 finished with value: 0.4588744588744588 and parameters: {'iterations': 186, 'learning_rate': 0.12671942076538364, 'depth': 9, 'l2_leaf_reg': 0.23981178357614202, 'auto_class_weights': 'SqrtBalanced'}. Best is trial 1 with value: 0.4588744588744588.
[I 2025-03-24 20:48:08,457] Trial 2 finished with value: 0.39826839826839827 and parameters: {'iterations': 210, 'learning_rate': 0.11353188662004474, 'depth': 8, 'l2_leaf_reg': 1.4732078580398624, 'auto_class_weights': 'SqrtBalanced'}. Best is trial 1 with value: 0.4588744588744588.
[I 2025-03-24 20:48:27,927] Tria

fold,▁▅█▁▅█▁▅▁▅█▁▅█▁▅▁▅█▁▅█▁▅▁▅█▁▅█▁▅▁▅█▁▅█▁█
fold_f1,▆▆▄█▇▄▇▅█▇▄▇▇▄▆██▇▄▇▇▄█▇█▇▄▁▅▃▆▇▇▇▄▇▇▄█▄
fold_precision,▆▅▆▇▆█▇▆█▆██▆█▄▅█▆█▇▆▇▇████▁██▅▅▆▅▆▆▅▆██
fold_recall,▅▅▃▆▅▃▅▄▆▅▃▅▅▃▆█▆▅▃▅▅▃▆▅▆▅▃▁▄▂▅▇▆▇▃▆▇▃▆▃
mean_f1,▆█▆█▇█▇▇██▁▇▇▇█
mean_precision,▂▆▅▇▇▁▇▅▇█▂▁▁▂█
mean_recall,▅▆▅▆▅█▆▆▆▆▁▇▇▇▆
fold,5
fold_f1,0.28571
fold_precision,1
fold_recall,0.16667


Train F1-Score: 0.730
Train Precision: 0.637
Train Recall: 0.855
Test F1-Score: 0.344
Test Precision: 1.000
Test Recall: 0.208


Скрины метрик на валидации (step здесь имеет смысл очередного набора параметров):
![](images/catboost_tuning/optuna/f1.png)
![](images/catboost_tuning/optuna/precision.png)
![](images/catboost_tuning/optuna/recall.png)

Скор на тесте стал еще хуже, чем у gridsearch. Но в данном случае это объясняется неудачным подбором гиперпараметров, так как скор на валидации тоже хуже, чем у gridsearch.

##### Ax

In [39]:
catboost_ax_config = [
    {
        "name": "iterations",
        "type": "range",
        "bounds": [50, 250],
        "value_type": "int"
    },
    {
        "name": "learning_rate",
        "type": "range",
        "bounds": [1e-3, 0.2],
        "log_scale": True
    },
    {
        "name": "depth",
        "type": "range",
        "bounds": [4, 12],
        "value_type": "int"
    },
    {
        "name": "l2_leaf_reg",
        "type": "range",
        "bounds": [1e-2, 10],
        "log_scale": True,
        "value_type": "float"
    },
    {
        "name": "auto_class_weights",
        "type": "choice",
        "values": ["Balanced", "SqrtBalanced"]
    }
]

with wandb.init(project="job-fake-prediction", 
          config={
              "model_type": "catboost_ax_tunning",
              "validation": "stratified_group_kfold",
              "k_folds": n_folds
          }):
    best_params, experiment = tune_hyperparameters_ax(
        X_train_cb, y_train, groups_train,
        model_class=cb_cls,
        param_config=catboost_ax_config,
        n_trials=15
    )



cb_model_ax = cb_cls(**best_params).fit(X_train_cb, y_train)
calc_and_print_metrics(cb_model_ax, X_train_cb, y_train, is_test=False)
calc_and_print_metrics(cb_model_ax, X_test_cb, y_test)

[INFO 03-24 21:17:44] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter learning_rate. If that is not the expected value type, you can explicitly specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 03-24 21:17:44] ax.service.utils.instantiation: Inferred value type of ParameterType.STRING for parameter auto_class_weights. If that is not the expected value type, you can explicitly specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
/home/aksenovan/my_env/lib/python3.10/site-packages/ax/service/utils/instantiation.py:258: AxParameterWarning: `is_ordered` is not specified for `ChoiceParameter` "auto_class_weights". Defaulting to `True`  since there are exactly two choices.. To override this behavior (or avoid this warning), specify `is_ordered` during `ChoiceParameter` construction. Note that choice parameters with exactly 2 choices are always considered ordered and that the user-supplied `is_ordered`

fold,▁▅█▁▅█▁▅▁▅█▁▅█▁▅▁▅█▁▅█▁▅▁▅█▁▅█▁▅▁▅█▁▅█▁█
fold_f1,▆█▄▁▇▂█▇▇▆▅█▇▄▁▆█▇▄█▅▄▆▆█▅▄▇▆▅█▇█▇▄█▇▄▆▄
fold_precision,▄▆▇▁██▇▆▆▄▇▇▆▆▁████▇▃▆██▇▃▆▆▅▇████████▄▆
fold_recall,▆█▃▁▆▂▇▆▇█▄▇▆▃▁▅▇▆▃▇▆▃▅▅▇▆▄▇▆▄▇▆▇▆▃▇▆▃▇▃
mean_f1,▇▂█▇█▁█▆▅▆▇███▆
mean_precision,▃▂▆▃▅▂█▂█▂▃███▁
mean_recall,▇▂▇█▇▁▇▇▄▇▇▇▇▇▇
fold,5
fold_f1,0.27273
fold_precision,0.75
fold_recall,0.16667


Train F1-Score: 0.702
Train Precision: 0.590
Train Recall: 0.867
Test F1-Score: 0.409
Test Precision: 1.000
Test Recall: 0.257


Скрины метрик на валидации (step здесь имеет смысл очередного набора параметров):
![](images/catboost_tuning/ax/f1.png)
![](images/catboost_tuning/ax/precision.png)
![](images/catboost_tuning/ax/recall.png)

Здесь скор на валидации получился чуть хуже, чем у gridsearch, а на тесте примерно такой же. Однако это по-прежнему хуже, чем у исходного катбуста, для которого не подбирались иперпараметры.

## Итоговый выбор модели

### Анализ результатов обучения

Мы обучили 4 модели:
- **SVM.** За счет нелинейности в ядре показал себя лучше бейзлайна на 17% и 22% до и после тюнинга гиперпараметров соотвественно.
- **Случайный лес.** Очень сильно переобучился, из-за чего получил скор ниже бейзлайна.
- **Catboost.** Нативная работа с текстовыми признаками помогла ему превзойти бейзлайн почти в два раза с f1-мерой равной 0.45. Однако тюнинг гиперпарметров не принес положительных результатов, вероятно, из-за переобучения под валидацию.
- **BERT+logreg.** Получил f1-меру на тесте равную 0.51, а на валидации -- лишь 0.12. Скорее всего, это связано с утечкой в данных, которую может уловить лишь достаточно сложная NLP-модель. Утечка, вероятно, связана с семантической близостью объявлений одного фродера.

### Выбор продовой модели

Из-за слишком низкого качества случайного леса и использования утечки бертом модель для прода нужно выбирать среди SVM и катбуста. Я выбираю catboost, так как:

- Catboost существенно превосходит SVM на тесте по всем метрикам (f1, precision, recall)
- В нашей задаче вполне допустим офлайн инференс, поэтому время работы нам менее важно, чем перформанс. Хотя в данном случае не очевидно, какая из моделей будет быстрее, ведь для SVM нужно подготовить дорогие с точки зрения вычислений признаки

Сохраним итоговую модель в файл

In [41]:
model_cb.save_model("catboost_fraud_model.cbm")

## Демо инференса модели

Инструкции по запуску демо находятся в файле [README.md](README.md)